# <font color="red">Hackday - Dados Inglórios</font>

## Planejamento da Solução

### Input

**Problema de Negócio**

A inclusão financeira continua a ser um dos principais obstáculos ao desenvolvimento econômico e humano na África. Por exemplo, no Quênia, Ruanda, Tanzânia e Uganda, apenas 9,1 milhões de adultos (14% dos adultos), têm acesso ou usam uma conta bancária comercial.

Em 2008, o nível de inclusão financeira na África Subsaariana era de pouco mais de 23%. Em 2018, esse número quase dobrou. No Togo, Mazamesso Assih, ministro de finanças, coordena uma estratégia de inclusão financeira com bancos parceiros. Ele afirma que garantir acesso a serviços financeiros básicos para a população, é fundamental para impulsionar a economia Africana.

Em 2022, conforme destacado no último relatório do Banco Central da África Ocidental, atingiu-se uma das mais altas taxas de inclusão financeira da região, próxima de 82%. Uma parcela significativa desse aumento veio do aumento dos serviços financeiros digitais.

Assih afirma também, que existem três razões principais pelas quais as nações africanas devem apostar na inclusão financeira:

 1. Tornar os serviços financeiros mais acessíveis promove o empoderamento das pessoas mais vulneráveis, especialmente as mulheres.
 2. Combater redes criminosas, passar de uma economia exclusivamente monetária para uma infraestrutura financeira digital torna mais fácil para as autoridades rastrear transações e lidar com contrabandistas e traficantes.
 3. impulsionará a conectividade dos setores público e privado para um crescimento sustentável, apoiando as start-ups africanas.
 
O objetivo desta competição é criar um modelo de aprendizado de máquina para prever quais indivíduos vão ou não utilizar uma conta bancária. Os modelos e soluções desenvolvidos devem fornecer uma indicação do estado da inclusão financeira no Quênia, Ruanda, Tanzânia e Uganda.



**Dados**

Os dados necessários podem ser encontrados no [repositório da atividade](https://www.kaggle.com/c/inclusao-financeira-na-africa). No arquivo train.csv você encontrará:

|Feature|Descrição|
|:------|:--------|
|country| Pais de Origem.
|year| Ano da Inclusão
|uniqueid| Id do Usuário.
|bank_account| Possui ou Não a Conta no Banco.
|location_type| Tipo de Localidade (Rural...)
|cellphone_access| Possui ou Não Celular.
|household_size| Tamanho da Casa.
|age_of_respondent| Idade do Usuário.
|gender_of_respondent| Gênero do Usuário.
|relationship_with_head| Status com o Dono da Conta.
|marital_status| Situação Civil.
|education_level| Nível de Educação.
|job_type| Tipo de Trabalho.

### Output

Um arquivo CSV com a previsão do modelo dizendo quais clientes abrirão uma conta digital em duas colunas:  

|uid|	bank_account|
|:--|:--------------|
|Rwanda_uniqueid_625|	No|
|Rwanda_uniqueid_1561|	No|
|Rwanda_uniqueid_4806|	No|
|Tanzania_uniqueid_4902|	Yes|
|Rwanda_uniqueid_980|	No|
|Rwanda_uniqueid_7043|	No|

### Tasks

- Descrição dos Dados
    - Renomear colunas
    - Checar dimenões
    - Checar tipos
    - Alterar tipos
    - Checar vazios
    - Preencher os vazios
    - Descrições Estatísticas

- Feature Engineering
    - Mapa de hipóteses
    - Criação de hipóteses
- Seleção de features (Filtro)
- EDA
    - Univariada
    - Bivariada (teste de hipóteses)
    - Multivariadas
- Preparação dos dados
    - Rescala das features
- Seleção de features
- Construção do modelo
    - modelos de classificação
    - guideline
- Avaliação do modelo
- Fine Tuning
- CSV Final

## 0. Settings

### 0.1. Imports

In [1]:
# manipulação de dados
import pandas as pd
import numpy as np

# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

# scikit-learn
from sklearn import model_selection as ms
from sklearn import ensemble
from sklearn import metrics

### 0.2. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container {width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()

jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


### 0.3. Data Loading

In [3]:
train_file = './data/train.csv'

df_raw = pd.read_csv(train_file)
df_raw.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,uid,bank_account
0,Rwanda,2016,uniqueid_4858,Rural,Yes,6,45,Male,Head of Household,Divorced/Seperated,Primary education,Farming and Fishing,Rwanda_uniqueid_4858,No
1,Tanzania,2017,uniqueid_3015,Urban,No,4,33,Female,Head of Household,Single/Never Married,Primary education,Self employed,Tanzania_uniqueid_3015,No
2,Rwanda,2016,uniqueid_103,Rural,Yes,7,43,Male,Head of Household,Married/Living together,Secondary education,Farming and Fishing,Rwanda_uniqueid_103,No
3,Rwanda,2016,uniqueid_4582,Rural,No,6,35,Female,Head of Household,Married/Living together,Primary education,Farming and Fishing,Rwanda_uniqueid_4582,No
4,Tanzania,2017,uniqueid_2854,Urban,Yes,2,30,Male,Head of Household,Single/Never Married,Primary education,Informally employed,Tanzania_uniqueid_2854,No


## 1. Descrição dos Dados

In [4]:
df1 = df_raw.copy()

In [12]:
df1['country'].unique()

array(['Rwanda', 'Tanzania', 'Kenya', 'Uganda'], dtype=object)

In [13]:
df1['location_type'].unique()

array(['Rural', 'Urban'], dtype=object)

In [14]:
df1['relationship_with_head'].unique()

array(['Head of Household', 'Spouse', 'Child', 'Parent', 'Other relative',
       'Other non-relatives'], dtype=object)

In [15]:
df1['marital_status'].unique()

array(['Divorced/Seperated', 'Single/Never Married',
       'Married/Living together', 'Widowed', 'Dont know'], dtype=object)

In [16]:
df1['education_level'].unique()

array(['Primary education', 'Secondary education',
       'Vocational/Specialised training', 'Tertiary education',
       'No formal education', 'Other/Dont know/RTA'], dtype=object)

### 1.1. Renomear features

### 1.2. Checar Dimensões

### 1.3. Checar Tipos

### 1.4. Alterar Tipos

### 1.5. Checa NaN

In [5]:
df1.isna().sum()

country                   0
year                      0
uniqueid                  0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
uid                       0
bank_account              0
dtype: int64

### 1.6. Preenche NaN

### 1.7. Estatística Descritiva

## 2. Feature Engineering

In [6]:
df2 = df1.copy()

In [11]:
df2.dtypes

country                   object
year                       int64
uniqueid                  object
location_type             object
cellphone_access          object
household_size             int64
age_of_respondent          int64
gender_of_respondent      object
relationship_with_head    object
marital_status            object
education_level           object
job_type                  object
uid                       object
bank_account              object
dtype: object

In [23]:
count = df2['country'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['country'] = df2['country'].map(mapper)

In [24]:
count = df2['location_type'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['location_type'] = df2['location_type'].map(mapper)

In [25]:
count = df2['cellphone_access'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['cellphone_access'] = df2['cellphone_access'].map(mapper)

In [26]:
count = df2['gender_of_respondent'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['gender_of_respondent'] = df2['gender_of_respondent'].map(mapper)

In [27]:
count = df2['relationship_with_head'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['relationship_with_head'] = df2['relationship_with_head'].map(mapper)

In [28]:
count = df2['marital_status'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['marital_status'] = df2['marital_status'].map(mapper)

In [29]:
count = df2['education_level'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['education_level'] = df2['education_level'].map(mapper)

In [30]:
count = df2['job_type'].value_counts()
mapper = dict()
for ind in count.index:
    mapper[ind] = count[ind]
    
df2['job_type'] = df2['job_type'].map(mapper)

In [31]:
df2['bank_account'] = df2['bank_account'].apply(lambda x: 1 if x == 'Yes' else 0)

### 2.1. Mapa de Hipóteses

### 2.2. Criação de Hipóteses

## 3. Seleção de Features

In [32]:
df3 = df2.copy()

## 4. EDA

In [33]:
df4 = df3.copy()

### 4.1. Univariada

### 4.2. Bivariadas

### 4.3. Multivariadas

## 5. Preparação dos Dados

In [34]:
df5 = df4.copy()

### 5.1. Reescala dos Dados

## 6. Seleção de Features

In [35]:
df6 = df5.copy()

### 6.1. Separação de Treino e Teste

In [36]:
df6.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,uid,bank_account
0,4335,2016,uniqueid_4858,7100,8786,6,45,4860,6358,1034,6408,2732,Rwanda_uniqueid_4858,0
1,3288,2017,uniqueid_3015,4662,2976,4,33,6902,6358,3970,6408,3207,Tanzania_uniqueid_3015,0
2,4335,2016,uniqueid_103,7100,8786,7,43,4860,6358,5433,2129,2732,Rwanda_uniqueid_103,0
3,4335,2016,uniqueid_4582,7100,2976,6,35,6902,6358,5433,6408,2732,Rwanda_uniqueid_4582,0
4,3288,2017,uniqueid_2854,4662,8786,2,30,4860,6358,3970,6408,2788,Tanzania_uniqueid_2854,0


In [37]:
X = df6.drop(['uniqueid','uid','bank_account'],axis=1)
y = df6['bank_account']

x_train, x_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.3, random_state=42)

## 7. Construção do Modelo

### 7.1. Random Forest Classifier

In [38]:
# criar o modelo
rfc_model = ensemble.RandomForestClassifier(random_state=42)

# treinar o modelo
rfc_model.fit(x_train, y_train)

# fazer a predição
yhat_rfc = rfc_model.predict(x_test)

# calcula a performance
metrics.f1_score(y_test, yhat_rfc)

0.4320388349514563

## 8. Avaliação do Modelo

## 9. Fine Tunning

## 10. Output da Submissão

In [60]:
x_test.head()

,year,household_size,age_of_respondent
3717,2016,6,48
11036,2016,3,27
2088,2017,4,32
1399,2018,3,19
1178,2017,2,36


In [61]:
y_test[0:5]

3717     0
11036    0
2088     0
1399     0
1178     0
Name: bank_account, dtype: int64